In [1]:
# Import necessary libraries
import os
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import DenseNet121
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

In [2]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
)

# Only rescale validation and test images
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [3]:
# Load training data with data augmentation
train_generator = train_datagen.flow_from_directory(
    r"Data\Training",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

# Load validation data
validation_generator = validation_datagen.flow_from_directory(
    r"Data\Validation",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    r"Data\Testing",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 40974 images belonging to 7 classes.
Found 8782 images belonging to 7 classes.
Found 8787 images belonging to 7 classes.


In [4]:
# Compute class weights to address data imbalance
train_dir = r"Data\Training"
class_labels = list(train_generator.class_indices.keys())  # Get class labels
class_counts = [len(os.listdir(os.path.join(train_dir, label))) for label in class_labels]  # Count files per class

In [5]:
# Compute class weights
class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(class_labels)),
    y=np.concatenate([[i] * count for i, count in enumerate(class_counts)])
)

class_weight_dict = dict(enumerate(class_weight))

In [6]:
# Define the base model (DenseNet121) for transfer learning
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [7]:
# Freeze the layers of the base model
base_model.trainable = False

In [8]:
# Add custom layers for classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.5)(x)

output_layer = Dense(len(class_labels), activation='softmax')(x)

In [9]:
# Create the final model
model = Model(inputs=base_model.input, outputs=output_layer)

In [10]:
# Compile the model with optimizer and loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [12]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

Epoch 1/25


InvalidArgumentError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: `generator` yielded an element that did not match the expected structure. The expected structure was (tf.float32, tf.float32, tf.float32), but the yielded element was (array([[[[0.01199831, 0.01199831, 0.01199831],
         [0.01383284, 0.01383284, 0.01383284],
         [0.01566737, 0.01566737, 0.01566737],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.0127882 , 0.0127882 , 0.0127882 ],
         [0.01095367, 0.01095367, 0.01095367],
         [0.00911914, 0.00911914, 0.00911914],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.01878735, 0.01878735, 0.01878735],
         [0.01787009, 0.01787009, 0.01787009]

In [ ]:
# Unfreeze last few layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-65]:
    layer.trainable = False

In [ ]:
# Recompile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32
)

In [ ]:
# Save the trained model
model.save('model.h5')

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy:.2f}")